In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
tqdm.pandas()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Bidirectional,Convolution1D,Dense,GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from gensim.models import KeyedVectors

In [0]:
df = pd.read_csv('/content/drive/My Drive/EXL Data/Company response to consumer.csv')
df.head()

,Consumer complaint narrative,Total Clean Text,After Stop Word Removal,Company response to consumer,Complaint ID
0,transworld systems inc. \nis trying to collect...,trans world systems inc be try to collect deb...,trans world systems inc try collect debt mine ...,Closed with explanation,3384392
1,I would like to request the suppression of the...,would like to request the suppression of the ...,would like request suppression follow items cr...,Closed with explanation,3417821
2,"Over the past 2 weeks, I have been receiving e...",over the past weeks have be receive excessiv...,past weeks receive excessive amount telephone ...,Closed with explanation,3433198
3,"I was sold access to an event digitally, of wh...",be sell access to an event digitally of which...,sell access event digitally screenshots detail...,Closed with explanation,3366475
4,While checking my credit report I noticed thre...,while check my credit report notice three col...,check credit report notice three collections c...,Closed with explanation,3385399


In [0]:
df['Company response to consumer'].value_counts()

Closed with explanation            385665
Closed with non-monetary relief     56652
Closed with monetary relief         26882
Closed                               3661
Untimely response                    3082
Name: Company response to consumer, dtype: int64

In [0]:
def renaming(text):
  if text == 'Closed with explanation':
    return 'Others'
  elif text == 'Closed':
    return 'Others'
  elif text == 'Untimely response':
    return 'Others'
  else:
    return text

In [0]:
df['Company response to consumer'] = df['Company response to consumer'].progress_apply(renaming)

100%|██████████| 475942/475942 [00:00<00:00, 1123288.51it/s]


In [0]:
#df.sort_values(by = 'Company response to consumer',ascending=True,inplace=True)

In [0]:
#df.head()

In [0]:
df['Company response to consumer'].value_counts()

Others                             392408
Closed with non-monetary relief     56652
Closed with monetary relief         26882
Name: Company response to consumer, dtype: int64

In [0]:
#df = df[:130000]
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Consumer complaint narrative,Total Clean Text,After Stop Word Removal,Company response to consumer,Complaint ID
0,XXXX XXXX XXXX XX/XX/XXXX XXXX XXXX XXXX XXXX...,ca trans union re acc to who...,ca trans union acc may concern letter regard n...,Others,2839519
1,I have unauthorized inquiries on my report tha...,have unauthorized inquiries on my report that...,unauthorized inquiries report trans union fail...,Others,2773546
2,I cancelled my debit card and ordered a new on...,cancel my debit card and order new one rece...,cancel debit card order new one receive tempor...,Closed with monetary relief,3387487
3,We are using JP Morgan Chase as our lenders fo...,we be use jp morgan chase as our lenders for ...,use jp morgan chase lenders home mortgage loan...,Others,2175831
4,I HAVE BEEN WITH XXXX XXXX XXXX SINCE XXXX AND...,have be with since and have always pay my...,since always pay balance full every month good...,Others,3185176


In [0]:
x = df['Total Clean Text']
y = df['Company response to consumer']

le = LabelEncoder()

y = le.fit_transform(y)
y = to_categorical(y)

In [0]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(x)

seq = tokenizer.texts_to_sequences(x)

pad_seq = pad_sequences(seq,maxlen = 500,padding='post',truncating='pre')

In [0]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

37592

In [0]:
embedding_values = {}
f = open('/content/drive/My Drive/Embeddings/glove.840B.300d.txt')
for line in tqdm(f):
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype = 'float')
  if coef is not None:
    embedding_values[word] = coef

In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Embeddings/GoogleNews-vectors-negative300.bin', \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Found 3000000 word vectors of word2vec


In [0]:
all_embds = np.stack(embedding_values.values())
mean,std = all_embds.mean(),all_embds.std()
mean,std

In [0]:
embedding_matrix = np.random.normal(mean,std,(vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
  value = embedding_values.get(word)
  if value is not None:
    embedding_matrix[i] = value

In [0]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Embedding(vocab_size,300,input_length=500,weights = [embedding_matrix],trainable = False))
model.add(GRU(32,return_sequences=False))
# model.add(Convolution1D(32,2,activation = 'relu'))
# model.add(GlobalMaxPool1D())
model.add(Dense(128,activation = 'relu'))
model.add(Dense(3,activation = 'softmax'))

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy',f1_m,precision_m,recall_m])

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(pad_seq,y,test_size = 0.15,random_state= 42)

In [0]:
model.fit(pad_seq,y,batch_size = 32,epochs = 7,validation_split = 0.15 )

Epoch 1/7
12643/12643 [==============================] - 258s 20ms/step - loss: 0.5468 - accuracy: 0.8246 - f1_m: 0.8230 - precision_m: 0.8242 - recall_m: 0.8222 - val_loss: 0.4839 - val_accuracy: 0.8246 - val_f1_m: 0.8244 - val_precision_m: 0.8257 - val_recall_m: 0.8233
Epoch 2/7
12643/12643 [==============================] - 255s 20ms/step - loss: 0.4700 - accuracy: 0.8260 - f1_m: 0.8253 - precision_m: 0.8292 - recall_m: 0.8216 - val_loss: 0.4618 - val_accuracy: 0.8270 - val_f1_m: 0.8263 - val_precision_m: 0.8317 - val_recall_m: 0.8212
Epoch 3/7
12643/12643 [==============================] - 259s 20ms/step - loss: 0.4540 - accuracy: 0.8283 - f1_m: 0.8275 - precision_m: 0.8318 - recall_m: 0.8234 - val_loss: 0.4529 - val_accuracy: 0.8294 - val_f1_m: 0.8286 - val_precision_m: 0.8335 - val_recall_m: 0.8240
Epoch 4/7
12643/12643 [==============================] - 257s 20ms/step - loss: 0.4455 - accuracy: 0.8299 - f1_m: 0.8291 - precision_m: 0.8334 - recall_m: 0.8250 - val_loss: 0.4502 - v

In [0]:
model.save('Consumer response Word2Vec 1000 Length.h5')

In [0]:
import pickle

# saving
with open('tokenizer_Consumer_response_Word2Vec_1000_Length.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
le.classes_

array(['Closed with monetary relief', 'Closed with non-monetary relief',
       'Others'], dtype=object)